In [1]:
import requests
from bs4 import BeautifulSoup as bs
import pandas as pd
import re

In [2]:
url = "https://www.espn.com/mens-college-basketball/matchup?gameId=" + str(401372480)

r = requests.get(url)

webpage = bs(r.content)

In [3]:
#create dictionary for CAL & opponent with respective scores
team_name = [name.string for name in webpage.find_all("td", attrs={"class", "team-name"})]
score = [float(final_score.string) for final_score in webpage.find_all("td", attrs={"class", "final-score"})]

team_score = dict(zip(team_name, score))

#determine opponent
for team in team_name:
    if team != "CAL":
        opponent = team

In [4]:
#locate table for main data regarding game
table = webpage.select("table.mod-data")[0]

#create column_names and locate the rows of the table

column_names = (list(team_score.keys()))
#columns = table.find_all("td", string=re.compile("[A-Za-z%]+"))
#column_names = [c.get_text().strip() for c in columns]

table_rows = table.find("tbody").find_all("tr")

#accumulate all the table_rows into one list
row_names, l = [], []
for tr in table_rows:
    td = tr.find_all("td")
    #row: [STAT_NAME, NUM1, NUM2]
    row = [tr.get_text().strip() for tr in td]
    #.append([NUM1, NUM2])
    l.append(row[1:])
    #.append([STAT_NAME])
    row_names.append(row[0])

In [5]:
#create data table with column_names and rows within table.mod-data
df = pd.DataFrame(l, columns=column_names)

In [6]:
#searches a string with - and seperates elements from either side into a list
def try_convert(val): 
    hyphen_index = val.index("-")
    return [val[:hyphen_index], val[hyphen_index + 1:]]

#append new rows for FGA, FGM, 3PTA, 3PTM, FTA, FTM for later calculations in MAKES-ATTEMPT format
for index in df.index:
    if re.search("-", df.loc[index, "CAL"]) and re.search("-", df.loc[index, opponent]):
        #multiple assignment with try_convert to extract attempts and makes 
        makes_cal, att_cal = try_convert(df.loc[index, "CAL"])[0], try_convert(df.loc[index, "CAL"])[1]
        makes_opp, att_opp = try_convert(df.loc[index, opponent])[0], try_convert(df.loc[index, opponent])[1]
        
        #assigns current row with makes
        df.loc[index, "CAL"], df.loc[index, opponent] = makes_cal, makes_opp
    
        #append new row with attempts, increasing index to reorder later
        df2 = pd.DataFrame({'CAL': att_cal, opponent: att_opp}, index=[index + .1])
        df = df.append(df2, ignore_index = False)

In [7]:
#reorder rows with increasing index
df = df.sort_index().reset_index(drop=True)

#adds FGA, FGM, 3PTA, 3PTM, FTA, FTM row names
row = []
def add_makes_att(lst, new_lst):
    for stat in lst:
        if re.search("FG|3PT|FT", stat):
            new_lst.append(stat + "M")
            new_lst.append(stat +"A")
        else:
            new_lst.append(stat)
    return new_lst

row_names = add_makes_att(row_names, row)

#set row indexes as STAT_NAME within row
df.index = row_names

In [8]:
#turns all stats into float
df["CAL"] = pd.to_numeric(df["CAL"], downcast="float")
df[opponent] = pd.to_numeric(df[opponent], downcast="float")
df

,CAL,UNLV
FGM,21.000000,21.0
FGA,57.000000,56.0
Field Goal %,36.799999,37.5
3PTM,4.000000,10.0
3PTA,14.000000,29.0
Three Point %,28.600000,34.5
FTM,6.000000,3.0
FTA,6.000000,5.0
Free Throw %,100.000000,60.0
Rebounds,38.000000,35.0
